# Lift demo

This notebook demonstrates:
- how to load a DiVinE model using the `%%dve` magic
- how to use Spot's functions to build a model checker in a few lines

In [5]:
from AAA_utils import *

## The model

In the following model, the global variable `req[i]` is `1` iff the elevator has been requested on floor `i`.
Variable `t` represents the *target* floor of the lift, and `p` its current *position*.
The `go` is a Boolean that the controller sets to instruct the cabin to move, and that the cabin reset upon reaching the target floor.

In [6]:
%%dve lift_model
byte req[4];
int t,p;
byte go;

process cabin
{
  state idle, mov, open;
  init idle;
  trans
    idle -> mov  {guard go>0;},
    mov -> open  {guard t==p;},
    mov -> mov   {guard t<p; effect p=p-1;},
    mov -> mov   {guard t>p; effect p=p+1;},
    open -> idle {effect req[p]=0,go=0;};
}

process environment
{
  state read;
  init read;
  trans
    read -> read {guard req[0]==0; effect req[0]=1;} ,
    read -> read {guard req[1]==0; effect req[1]=1;} ,
    read -> read {guard req[2]==0; effect req[2]=1;} ,
    read -> read {guard req[3]==0; effect req[3]=1;};
}

process controller
{
  state work;
  init work;
  trans
    work -> work {guard req[0]==1 && go==0; effect t=0, go=1;},
    work -> work {guard req[1]==1 && go==0; effect t=1, go=1;},
    work -> work {guard req[2]==1 && go==0; effect t=2, go=1;},
    work -> work {guard req[3]==1 && go==0; effect t=3, go=1;};
}

/*
process controller
{
  byte ldir;
  state wait, work, done;
  init wait;
  trans
    wait -> work {guard go==0; effect t=t+(2*ldir)-1;},
    work -> wait {guard t<0 || t==4; effect ldir=1-ldir;},
    work -> done {guard t>=0 && t<4 && req[t]==1;},
    work -> work {guard t>=0 && t<4 && req[t]==0; effect t=t+(2*ldir)-1;},
    done -> wait {effect go=1;};
}
*/

system async;

UsageError: Cell magic `%%dve` not found.


Evaluating the above cell has compiled and loaded the model.  The argument of `%%dve` is the name of a variable that serves as a handle to that model.  Printing this variable will list all global variables 

In [4]:
lift_model

NameError: name 'lift_model' is not defined

To build a state space, we can use the `kripke()` method.  The argument is a list of atomic propositions (involving the model's variables) we want to use to label the Kripke structure.  If we just want to count the number of reachable states, we can just pass an empty list.

In [ ]:
k = lift_model.kripke([])
spot.stats_reachable(k).states

Using the specialized `lift_display()`, we can explore the start of the state space.  Do not try to display to many states, because the graph layout (done by GraphViz) will be very time consuming.

In [ ]:
@interact(n=IntSlider(1, 1, 150, 5))
def uncover_state_space(n):
    return lift_display(k, n)

## Model checking, step by step

The LTL property we want to check is that if whenever (`G`) the lift is called on the second floor (numbering is 0-based, so this is `req[1]`), then eventually (`F`) the lift will reach thich floor (`p==0`) and open its doors (`cabin.open`).

In [ ]:
f = spot.formula('G("req[1]" -> F("p==1" && "cabin.open"))')

We can collect the atomic propositions needed by this property:

In [ ]:
aplist = spot.atomic_prop_collect(f)
aplist

An now we build a state space labeled by those atomic propositions:

In [ ]:
k = lift_model.kripke(aplist)
k.show('.A<12')

The above "Kripke structure" can be interpreted as an automaton by pushing labels onto the outgoing transitions
(in Spot, this is just a rendering option; in practice behaving like an automaton is achieved by satisfying the OO interface for automata).

In [ ]:
k.show('.1AK<12')

Similarly, the negation of the LTL property can be translated into an automaton that recognize all behaviours we do not want to see in our model.

In [ ]:
aut = spot.formula_Not(f).translate()
aut

And then we just have to check if there exists a `run` of `k` that intersects `aut`.

In [ ]:
run = k.intersecting_run(aut)
print(run)


We can also display this run graphically:

In [ ]:
lift_display(run)

## A model checker in 5 lines

Putting this all together, here is a function that takes an LTSmin model and a formula, and that returns a counterexample if one exists.

In [ ]:
def model_check(model, f):
    f = spot.formula(f)
    ss = model.kripke(spot.atomic_prop_collect(f))
    nf = spot.formula.Not(f).translate()
    return ss.intersecting_run(nf)

In [ ]:
run = model_check(lift_model, 'G("req[1]" -> F("p==1" && "cabin.open"))')
lift_display(run)

Now you could go back to the model, and change the controler to get rid of that scenario.